## 準備

マルコフ決定過程 (MDP):  $\mathcal{M}=(\mathcal{S}, \mathcal{A}, \mathcal{P}, r, \rho, \gamma)$
* 状態空間:$\mathcal{S}$
* 行動空間:$\mathcal{A}$ 
* 状態遷移密度: $\mathcal{P}: \mathcal{S} \times \mathcal{A} \times \mathcal{S} \rightarrow \mathbb{R}_{+}$
* 報酬関数: $r: \mathcal{S} \times \mathcal{A} \rightarrow\left[-r_{\max }, r_{\max }\right]$
* 割引率: $\gamma \in(0,1)$
* $r_{\max} > 0$ 
* $\rho$ (初期状態分布)
* $H$ (軌跡長)

学習設定
* 方策: $\pi_\theta(\cdot \mid s)$  (where $\theta \in \mathbb{R}^d$ and $s \in \mathcal{S}$)
* 軌跡:$$\tau=\left(s_0, a_0, \cdots, s_{H-1}, a_{H-1}\right)$$
* 軌跡の確率:
$$
p\left(\tau \mid \pi_\theta\right) \stackrel{\text { def }}{=} \rho\left(s_0\right) \pi_\theta\left(a_0 \mid s_0\right) \prod_{t=1}^{H-1} \mathcal{P}\left(s_t \mid s_{t-1}, a_{t-1}\right) \pi_\theta\left(a_t \mid s_t\right)
$$
* 期待収益(目的関数): $J(\theta) \stackrel{\text { def }}{=} \mathbb{E}_{\rho, \pi_\theta}\left[\sum_{t=0}^{+\infty} \gamma^t r\left(s_t, a_t\right)\right]$
* 有限期待収益: $J_H(\theta) \stackrel{\text { def }}{=} \mathbb{E}_{\rho, \pi_\theta}\left[\sum_{t=0}^{H-1} \gamma^t r\left(s_t, a_t\right)\right]$
* 期待収益(展開):総報酬(確率変数)と総報酬が得られる確率を用いる
$$
J(\theta) = \sum_{s_0 \in \mathcal{S}} \sum_{a_0 \in \mathcal{A}}  ... \left[ \sum_{t=0}^{\infty} \gamma^t r(s_t, a_t) \right]  P(s_0, a_0, s_1, a_1, ...)
$$
ここで、期待収益が得られる確率は、	
$$
P(s_0, a_0, s_1, a_1, ...) = \rho(s_0) \pi_\theta(a_0|s_0) P(s_1|s_0, a_0) \pi_\theta(a_1|s_1) P(s_2|s_1, a_1) ...
$$
であり、$p\left(\tau \mid \pi_\theta\right)$の無限版と解釈できる。
	報酬関数は確率変数：状態および行動が$\rho$、$\pi$と$P$からサンプルされるので$s_t$、$a_t$が確率変数であるのだが、それを入力とする$r$はボレル可測関数であるため確率変数として扱える。また、確率変数の無限和は、有限値に収束する場合(割引無限とか)は、確率変数として扱えるので、総報酬も確率変数になる。
## 方策勾配とヘッセ行列

一次
* 有限期待収益の勾配:

$$
\nabla J_H(\theta)=\mathbb{E}_{\rho, \pi_\theta}[g(\tau, \theta)]
$$

	
なお、
$$
g(\tau, \theta)=\sum_{t=0}^{H-1}\left(\sum_{h=t}^{H-1} \gamma^h r\left(s_h, a_h\right)\right) \nabla \log \pi_\theta\left(a_t \mid s_t\right)
$$
である。
* 展開
	有限期待収益は、軌跡の確率
	$$J_H(\theta) = \mathbb{E}_{\rho, \pi_\theta}\left[\sum_{t=0}^{H-1} \gamma^t r(s_t, a_t)\right]$$
	期待収益の期待値は、$p\left(\tau \mid \pi_\theta\right)$を用いて表せるので、
	$$
	\sum_\tau = \sum_{s_0 \in \mathcal{S}} \sum_{a_0 \in \mathcal{A}} \sum_{s_1 \in \mathcal{S}} \sum_{a_1 \in \mathcal{A}} ... \sum_{s_{H-1} \in \mathcal{S}} \sum_{a_{H-1} \in \mathcal{A}}
	$$
	とおくと、
	$$
	J_H(\theta) = \sum_{\tau} p(\tau|\pi_\theta) \left(\sum_{t=0}^{H-1} \gamma^t r(s_t, a_t)\right)
	$$
	と書ける。これを用いて、$\nabla J_H(\theta)$ を計算すると、

	$$
	\nabla J_H(\theta) = \sum_{\tau} \nabla p(\tau|\pi_\theta) \left(\sum_{t=0}^{H-1} \gamma^t r(s_t, a_t)\right)
	$$
	ここで、対数微分 $\nabla \log f(x)= \frac{ \nabla f(x)}{f(x)}$ を用いると、
	$$
	\nabla p(\tau|\pi_\theta) = p(\tau|\pi_\theta) \nabla \log p(\tau|\pi_\theta)
	$$

	$\nabla \log p(\tau|\pi_\theta)$について、
	
	$$
	\begin{aligned}
	& \nabla \log p(\tau|\pi_\theta) \\
	& = \nabla \log\left(\rho\left(s_0\right) \pi_\theta\left(a_0 \mid s_0\right) \prod_{t=1}^{H-1} P\left(s_t \mid s_{t-1}, a_{t-1}\right) \pi_\theta\left(a_t \mid s_t\right)\right) \\
	& = \nabla \left[ \log \rho(s_0) + \sum_{t=0}^{H-1} \log \pi_\theta(a_t|s_t) + \sum_{t=0}^{H-1} \log P(s_{t+1}|s_t, a_t) \right] \\
	
	\end{aligned}
	$$
	であり、$\nabla$は$\theta$に関しての勾配なので、右辺の第1項と第2項は0となり、
	$$
	\nabla \log p(\tau|\pi_\theta) = \sum_{t=0}^{H-1} \nabla \log \pi_\theta(a_t|s_t)
	$$
	よって、
	$$
	\nabla J_H(\theta) = \sum_{\tau} p(\tau|\pi_\theta) \left(\sum_{t=0}^{H-1} \nabla \log \pi_\theta(a_t|s_t)\right) \left(\sum_{t=0}^{H-1} \gamma^t r(s_t, a_t)\right)
	$$
	
	ここで、$g(\tau, \theta) = \left(\sum_{t=0}^{H-1} \nabla \log \pi_\theta(a_t|s_t)\right) \left(\sum_{t=0}^{H-1} \gamma^t r(s_t, a_t)\right)$ としたいが、$g$は、$\sum_{t=0}^{H-1} \gamma^t r(s_t, a_t)$ではなく、$\sum_{h=t}^{H-1} \gamma^h r\left(s_h, a_h\right)$である。$t$以降のみ考慮するのは、行動 $a_{t}$ が報酬に影響を与えるのは$t$以降だけだからである。

二次:
$$
\begin{aligned}
\nabla^2 J_H(\theta) & =\mathbb{E}_{\rho, \pi_\theta}[B(\tau, \theta)], \\
B(\tau, \theta) & \stackrel{\text { def }}{=} \nabla \Phi(\tau, \theta) \nabla \log p\left(\tau \mid \pi_\theta\right)^T+\nabla^2 \Phi(\tau, \theta) \\
\Phi(\tau, \theta) &\stackrel{\text { def }}{=} \sum_{t=0}^{H-1}\left(\sum_{h=t}^{H-1} \gamma^h r\left(s_h, a_h\right)\right) \log \pi_\theta\left(a_t \mid s_t\right)
\end{aligned}
$$


## 正規化モーメンタムを用いた方策勾配アルゴリズム(N-PG-IGT)
![[Pasted image 20250223003156.png]]

## ヘッセ行列を用いた再帰的方策勾配アルゴリズム((N)-HARPG)
![[Pasted image 20250223122217.png]]

### 収束解析
#### 解析のための準備
##### 仮定1(方策パラメータの正則性)
任意の$s, a \in \mathcal{S} \times \mathcal{A}$及び$\theta \in \mathbb{R}^d$に対して以下が成り立つと仮定する。
* 関数$\theta \mapsto \pi_\theta(a \mid s)$は、正かつ連続かつ二階微分まで微分可能
* 一次、二次勾配の大きさは有限($\left\|\nabla \log \pi_\theta(a \mid s)\right\| \leq M_g$、$\left\|\nabla^2 \log \pi_\theta(a \mid s)\right\| \leq M_h$を満たすような$M_g>0$、$M_h>0$が存在する)
##### 命題1
* 関数$J$ は$L_g$-smoothかつ $L_g=\frac{r_{\max }\left(M_g^2+M_h\right)}{(1-\gamma)^2}$.
* $\mathbb{E}\left[\left\|g(\tau \mid \theta)-\nabla J_H(\theta)\right\|^2\right] \leq \sigma_g^2$ かつ $\sigma_g^2=\frac{r_{\max }^2 M_g^2}{(1-\gamma)^3}$.
	MSE?
$$
\begin{aligned}
\mathbb{E}\left[\left\|g(\tau \mid \theta)-\nabla J_H(\theta)\right\|^2\right] &= \mathbb{E}\left[\left(g(\tau \mid \theta)-\nabla J_H(\theta)\right) \cdot \left(g(\tau \mid \theta)-\nabla J_H(\theta)\right)\right]\\
&= \mathbb{E}\left[\|g(\tau \mid \theta)\|^2 - 2g(\tau \mid \theta) \cdot \nabla J_H(\theta) + \|\nabla J_H(\theta)\|^2\right]\\
&= \mathbb{E}\left[\|g(\tau \mid \theta)\|^2\right] - 2\mathbb{E}\left[g(\tau \mid \theta)\right] \cdot \nabla J_H(\theta) + \mathbb{E}\left[\|\nabla J_H(\theta)\|^2\right]\\
&= \mathbb{E}\left[\|g(\tau \mid \theta)\|^2\right] - 2\nabla J_H(\theta) \cdot \nabla J_H(\theta) + \|\nabla J_H(\theta)\|^2\\
&= \mathbb{E}\left[\|g(\tau \mid \theta)\|^2\right] - \|\nabla J_H(\theta)\|^2
\end{aligned}
$$
* $\mathbb{E}\left[\left\|B(\tau \mid \theta)-\nabla^2 J_H(\theta)\right\|^2\right] \leq \sigma_h^2$ かつ $\sigma_h^2=$ $\frac{r_{\max }^2\left(H^2 M_g^4+M_h^2\right)}{(1-\gamma)^4}$ (一つ上の式が成立したときに成立)
##### 仮定2(方策とその二次対数微分は$L$-smooth)
**仮定の仮定**
* 任意の$s, a \in \mathcal{S} \times \mathcal{A}$及び$\theta \in \mathbb{R}^d$に対して以下が成り立つ
* 関数$\theta \mapsto \pi_\theta(a \mid s)$は、正かつ連続かつ二階微分まで微分可能
任意の$\theta,\theta'$について、以下を満たす任意の$l_2 > 0$が存在する。
$$
\left\|\nabla^2 \log \pi_\theta(a \mid s)-\nabla^2 \log \pi_{\theta^{\prime}}(a \mid s)\right\| \leq l_2\left\|\theta-\theta^{\prime}\right\|
$$
##### 補題1(期待収益は$L_h$-リプシッツ連続)
仮定1と仮定2が成立するとする。
任意の$\theta,\theta'$について、
$$
\theta, \theta^{\prime} \in \mathbb{R}^d,\left\|\nabla^2 J(\theta)-\nabla^2 J\left(\theta^{\prime}\right)\right\| \leq L_h\left\|\theta-\theta^{\prime}\right\|
$$
であり、$L_h=\mathcal{O}\left((1-\gamma)^{-3}\right)$である。

##### 仮定3(Fisher-non-degenerate parameterized policy)

**定義:$\pi$により誘発されたフィッシャー情報行列**
$$
F_\rho(\theta) \stackrel{\text { def }}{=} \mathbb{E}_{s \sim d_\rho^{\pi_\theta}, a \sim \pi_\theta(\cdot \mid s)}\left[\nabla \log \pi_\theta(a \mid s) \nabla \log \pi_\theta(a \mid s)^{\top}\right]
$$
なお、
$$
d_\rho^{\pi_\theta}(\cdot) \stackrel{\text { def }}{=}(1-\gamma) \sum_{t=0}^{\infty} \gamma^t \mathbb{P}_{\rho, \pi_\theta}\left(s_t \in \cdot\right)
$$
**展開**
$$
F_\rho(\theta) \stackrel{\text { def }}{=}\sum_{s \in \mathcal{S}} d_\rho^{\pi_\theta}(s) \sum_{a \in \mathcal{A}} \pi_\theta(a|s) \nabla \log \pi_\theta(a | s) \nabla \log \pi_\theta(a | s)^{\top}
$$

##### 仮定3(Fisher-non-degenerate policy)

全ての$\theta$に対して、$F_\rho(\theta) \succcurlyeq \mu_F \cdot I_d$になるような固有値$\mu_F > 0$が存在する。
* 逆行列計算できる

##### 仮定4
全ての$\theta$について、trasnfer errorが以下になるような、$\epsilon_{\text{bias}} \geq 0$が存在する
$$
\mathbb{E}\left[\left(A^{\pi_\theta}(s, a)-(1-\gamma) w^*(\theta)^{\top} \nabla \log \pi_\theta(a \mid s)\right)^2\right] \leq \varepsilon_{\text {bias }}
$$
*  **$A^{\pi_\theta}(s, a)$**: アドバンテージ関数 
$$
A^\pi(s, a) \stackrel{\text { def }}{=} Q^\pi(s, a)-V^\pi(s)
$$ 

$\pi_{\theta}$が$A^{\pi_\theta}(s, a)$をいい感じに$(1-\gamma) w^*(\theta)^{\top} \nabla \log \pi_\theta(a \mid s))^2$で近似できる

##### 補題2(Relaxed弱いGradient domination)
仮定1、仮定3、仮定４が成立する場合、
$$
\forall \theta \in \mathbb{R}^d, \quad \varepsilon^{\prime}+\|\nabla J(\theta)\| \geq \sqrt{2 \mu}\left(J^*-J(\theta)\right)
$$
なお、

$$
\text { where } \varepsilon^{\prime}=\frac{\mu_F \sqrt{\varepsilon_{\text {bias }}}}{M_o(1-\gamma)} \text { and } \mu=\frac{\mu_F^2}{2 M^2}
$$
#### 定理1(N-PG-IGTのグローバル収束)
仮定1、仮定2、仮定3、仮定4が成立とする。$\gamma_t=\frac{6 M_g}{\mu_F(t+2)}, \eta_t=\left(\frac{2}{t+2}\right)^{4 / 5}$、$\eta_t=\left(\frac{2}{t+2}\right)^{4 / 5}$とし、$H=(1-\gamma)^{-1} \log (T+1)$とする。
このとき、任意の$T \geq 1$について、以下が成り立つ。
$$
J^*-\mathbb{E}\left[J\left(\theta_T\right)\right] \leq \mathcal{O}\left(\frac{\sigma_g+L_h}{(T+1)^{2 / 5}}\right)+\frac{\sqrt{\varepsilon_{\text {bias }}}}{1-\gamma}
$$
#### 定理1の証明



#### 定理2(HARPGのグローバル収束)
仮定1、仮定3、仮定4が成り立つとする。$\gamma_t = \gamma_0 \eta_t^{1/2}$, $\eta_t = \frac{2}{t+2}$ とし、ここで $\gamma_0 = \min \left\{ \frac{1}{8\sqrt{6}(L_g + \sigma_g + D_h \gamma^H)}, \frac{\sqrt{2} M_g}{\sqrt{3} \sigma_g \mu_F} \right\}$, $H = (1-\gamma)^{-1} \log(T+1)$ とする。このとき、任意の $T \geq 1$ に対して、以下が成り立つ。

$$
J^* - \mathbb{E}[J(\theta_T)] \leq \mathcal{O}\left( \frac{\sigma_g + L_g + \sigma_h}{\sqrt{T+1}} \right) + \frac{2\sqrt{\varepsilon_{\text{bias}}}}{1-\gamma}
$$



#### 定理3((N)-HARPGのグローバル収束)
仮定1、仮定3、仮定4が成り立つとする。$\gamma_t = \frac{6M_g}{\mu_F(t+2)}$, $\eta_t = \frac{2}{t+2}$, $H = (1-\gamma)^{-1} \log(T+1)$ とする。このとき、任意の $T \geq 1$ に対して、以下が成り立つ。

$$
J^* - \mathbb{E}[J(\theta_T)] \leq \mathcal{O}\left( \frac{\sigma_g + L_g + \sigma_h}{\sqrt{T+1}} \right) + \frac{\sqrt{\varepsilon_{\text{bias}}}}{1-\gamma}
$$




